In [9]:
#import necessary libraries
import numpy as np
from scipy.sparse import csr_matrix
from pathlib import Path
import sys
import pandas as pd
import torch
import collections
import numpy as np
import torch.nn as nn
import os
import argparse
sys.path.append("../../default-approach")
from data_processing.articles import Articles
from models.models import InnerProduct
import data_processing.dictionaries as dictionary
from pathlib import Path
import json
from pandas import json_normalize
import scipy
import boto3

dict_dir = Path("../../../data/dictionaries")
final_word_ids,final_url_ids, final_publication_ids = dictionary.load_dictionaries(dict_dir)
print("Dictionaries loaded.")

data_path = Path('../../../data/UI_feeds/scrape_data/browser-rss-articles-info.json')
dataset = Articles(data_path)
print("Data loaded.")

abs_model_path = Path("../../../data/gridsearch_results/2020-05-25/optimizer_type=RMS_use_all_words=False_emb_size=100_learning_rate=0.0001_word_embedding_type=mean/model/mean-inner-product-model.pt")
kwargs = dict(n_publications=len(final_publication_ids),
              n_articles=len(final_url_ids),
              n_attributes=len(final_word_ids),
              emb_size=100,
              sparse=False,
              use_article_emb=False,
              mode='mean')
model = InnerProduct(**kwargs)
model.load_state_dict(torch.load(abs_model_path))
print("Model Loaded.")

dataset.tokenize()
proper_data = dataset.map_items(final_word_ids,
                    final_url_ids,
                    final_publication_ids,
                    filter=True,
                    min_length=400)

data_path = Path("/users/rohan/news-classification/Data/feed_ranks/data")
if not data_path.is_dir():
    data_path.mkdir()
mapped_data_path = data_path / "mapped-data"
if not mapped_data_path.is_dir():
    mapped_data_path.mkdir()
train_mapped_path = mapped_data_path / "mapped_dataset.json"
with open(train_mapped_path, "w") as file:
    json.dump(proper_data, file)
raw_data = Articles(train_mapped_path)
print("Final: ", len(raw_data))
print(f"Filtered, Mapped Data saved to {mapped_data_path} directory")
print("-------------------")

word_articles = scipy.sparse.csr_matrix((len(raw_data), len(final_word_ids)), dtype=np.float32).toarray()

for idx, item in enumerate(raw_data.examples):
    item['text'] = list(set(item['text']))
    for entry in item['text']:
        word_articles[idx][entry] = 1

publication_emb = model.publication_embeddings.weight.data[0].cpu().numpy()
publication_bias = model.publication_bias.weight.data[0].cpu().numpy()
word_emb = model.attribute_emb_sum.weight.data.cpu().numpy()
word_bias = model.attribute_bias_sum.weight.data.cpu().numpy()
scipy.sparse.save_npz("/users/rohan/news-classification/Data/demo-data/sets/csr_articles_browser.npz", csr_matrix(word_articles))
print("Article-Word Matrix Saved")

'''
np.save("word_emb.npy", word_emb)
print("Word Embeddings Saved")

np.save("word_bias.npy", word_bias)
print("Word Biases Saved")

with open("/users/rohan/news-classification/Data/feed_ranks/data/mapped-data/mapped_dataset.json",'r') as file:
    real_data = json.load(file)
    
with open('/users/rohan/news-classification/data/dictionaries/reversed_word_ids.json', "r") as file:
    id_to_word = json.load(file)
'''

Dictionaries loaded.
Data loaded.
Model Loaded.
Final:  8234
Filtered, Mapped Data saved to \users\rohan\news-classification\Data\feed_ranks\data\mapped-data directory
-------------------
Article-Word Matrix Saved


'\nnp.save("word_emb.npy", word_emb)\nprint("Word Embeddings Saved")\n\nnp.save("word_bias.npy", word_bias)\nprint("Word Biases Saved")\n\nwith open("/users/rohan/news-classification/Data/feed_ranks/data/mapped-data/mapped_dataset.json",\'r\') as file:\n    real_data = json.load(file)\n    \nwith open(\'/users/rohan/news-classification/data/dictionaries/reversed_word_ids.json\', "r") as file:\n    id_to_word = json.load(file)\n'

In [10]:
import numpy as np
from scipy.sparse import csr_matrix
import time
import json
import scipy

publication_emb = np.asarray([1.0440499, 1.0030843, 1.0340449, 0.992087, 1.0509816, 1.0315005, -1.0493797, -1.0198538, 0.9712321, -1.026394, 
            -0.9687971, 1.0592866, -1.0200703, -1.0423145, 0.9929519, 1.0220934, 1.021279, -1.0265925, 0.9601833, 0.9763889, 
            1.0109168, -0.9728226, 0.97199583, -1.0237931, -0.9996001, 0.9932069, 0.97966635, -0.98893607, -0.9876815, -0.98812914, 
            -0.9625895, 0.99879754, 0.9876508, -0.9581506, -0.95436096, -0.9601925, -1.0134513, -0.98763955, 0.98665, -1.0140482, 
            1.004904, 0.9894275, -1.0044671, -0.9839679, -0.97082543, -0.9798079, 0.9926766, -0.97317344, 0.9797, -0.97642475, 
            -0.99420726, -0.9972062, -1.0104703, 1.0575777, 0.9957696, -1.0413874, -1.0056863, -1.0151271, -0.99969465, 0.97463423, 
            -0.98398715, -1.0211866, -1.0128828, -1.0024365, -0.9800189, 1.0457181, 1.0155835, -1.036794, -1.013707, -1.0498024, 
            -1.0252678, -1.0388161, -0.97501564, 0.97687274, 0.97906756, 1.0536852, 1.0590494, -0.96917725, 1.0247189, -0.9818878, 
            -1.0417286, -1.0204054, -1.0285249, -1.0329671, 0.9705739, 0.96375024, 0.9891868, 0.9892464, 1.039075, 1.0042666,
            0.9786834, 1.0199072, 0.98080486, 0.9698635, -0.99322844, -0.95841753, -0.99150276, 0.97394156, 0.9976019, -1.0375009], dtype=np.float32)

publication_bias = 0.99557
word_articles = scipy.sparse.load_npz('/users/rohan/news-classification/Data/demo-data/sets/csr_articles_browser.npz')
word_emb = np.load('/users/rohan/news-classification/Data/demo-data/word_emb.npy')
word_bias = np.load('/users/rohan/news-classification/Data/demo-data/word_bias.npy')

with open("/users/rohan/news-classification/Data/feed_ranks/data/mapped-data/mapped_dataset.json",'r') as file:
    real_data = json.load(file)
    
with open('/users/rohan/news-classification/data/dictionaries/reversed_word_ids.json', "r") as file:
    id_to_word = json.load(file)
print("Data Loaded Successfully!")

time1 = time.time()
article_embeddings = word_articles.dot(word_emb)

emb_times_publication = np.dot(article_embeddings, publication_emb.reshape(100,1))

article_bias = word_articles.dot(word_bias)

product_with_bias = emb_times_publication + article_bias

word_counts = word_articles.sum(axis=1).reshape(word_articles.shape[0], 1)

final_logits = np.divide(product_with_bias, word_counts) + float(publication_bias)

indices = final_logits.argsort(axis=0)[-75:].reshape(75)

word_logits = np.dot(word_emb, publication_emb.reshape(100,1)) + word_bias

top_articles = word_articles[indices.tolist()[0]]

broadcasted_words_per_article = top_articles.toarray() * word_logits.T

sorted_word_indices = broadcasted_words_per_article.argsort(axis=1)

return_articles = []

i=0
for idx in indices.tolist()[0]:
    current_article = real_data[int(idx)]
    current_article['logit'] = float(final_logits[int(idx)])
    current_sorted_words = sorted_word_indices[i]
    top_words = []
    least_words = []
    for top_word in current_sorted_words[-10:]:
        word = id_to_word[str(top_word)]
        top_words.append(word)
    for least_word in current_sorted_words[:10]:
        word = id_to_word[str(least_word)]
        least_words.append(word)
    current_article['top_words'] = top_words
    current_article['least_words'] = least_words
    return_articles.append(current_article)
    i+=1
    
time2 = time.time()

print(time2-time1)

Data Loaded Successfully!
0.9528157711029053


In [11]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

ordered_return_articles = return_articles[::-1]
grand_html = []
for idx, article in enumerate(ordered_return_articles):
    grand_html.append("<tr>")
    grand_html.append(f"<td class=\"title mdl-data-table__cell--non-numeric\"><a class=\"article_link\" href=\"{article['link']}\">{article['title']}</a>")
    grand_html.append("<br></br>")
    grand_html.append("<br></br>")
    top_word_list = ""
    for item in article['top_words']:
        if len(item) > 2 and not RepresentsInt(item):
            top_word_list += item
            top_word_list += ", "
    least_word_list = ""
    for item in article['least_words']:
        if len(item) > 2:
            least_word_list += item
            least_word_list += ", "
    grand_html.append(f"<p class=\"top_words\">{top_word_list[:-2]}</p>")
    grand_html.append(f"<p class=\"least_words\">{least_word_list[:-2]}</p>")
    grand_html.append("</td>")
    grand_html.append(f"<td class=\"publication\">{article['publication']}</td>")
    grand_html.append(f"<td class=\"logit\">{article['logit']}</td>")
    grand_html.append("</tr>")
print("\n".join(grand_html))

<tr>
<td class="title mdl-data-table__cell--non-numeric"><a class="article_link" href="http://www.colinmcginn.net/mind-dependence/">Mind-Dependence</a>
<br></br>
<br></br>
<p class="top_words">finite, thus, neurons, notion, hence, indeed, attractive, suppose, moreover</p>
<p class="least_words">isn, says, stemming, including, also, appearance, arguing</p>
</td>
<td class="publication">browser</td>
<td class="logit">7.9061784744262695</td>
</tr>
<tr>
<td class="title mdl-data-table__cell--non-numeric"><a class="article_link" href="https://www.simonsfoundation.org/2020/05/06/finding-prime-locations-the-continuing-challenge-to-prove-the-riemann-hypothesis/">Finding Prime Locations: The Continuing Challenge to Prove the Riemann Hypothesis</a>
<br></br>
<br></br>
<p class="top_words">hypothesis, consisting, conjecture, behavior, graph, whereas, wikimedia, domain, credit</p>
<p class="least_words">above, key, gives, series, makes, suggests, than, friedrich, 1700s</p>
</td>
<td class="publica

In [113]:
#import necessary libraries
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from pathlib import Path
import sys
sys.path.append("..")
from models.models import InnerProduct
import pandas as pd
import torch
import collections
import numpy as np
import torch.nn as nn
import os
import argparse
from data_processing.articles import Articles
from models.models import InnerProduct
import data_processing.dictionaries as dictionary
from pathlib import Path
import json
from pandas import json_normalize
from scipy import sparse
import boto3

dict_dir = Path("../../data/dictionaries")
final_word_ids,final_url_ids, final_publication_ids = dictionary.load_dictionaries(dict_dir)
print("Dictionaries loaded.")

data_path = Path('../../data/final-data/train.json')
dataset = Articles(data_path)
print("Data loaded.")

dataset.tokenize()
proper_data = dataset.map_items(final_word_ids,
                    final_url_ids,
                    final_publication_ids,
                    filter=True,
                    min_length=200)

data_path = Path("/users/rohan/news-classification/Data/embedding-search/data")
if not data_path.is_dir():
    data_path.mkdir()
mapped_data_path = data_path / "mapped-data"
if not mapped_data_path.is_dir():
    mapped_data_path.mkdir()
train_mapped_path = mapped_data_path / "mapped_dataset.json"
with open(train_mapped_path, "w") as file:
    json.dump(proper_data, file)
raw_data = Articles(train_mapped_path)
print("Final: ", len(raw_data))
print(f"Filtered, Mapped Data saved to {mapped_data_path} directory")
print("-------------------")

abs_model_path = Path("../../data/gridsearch_results/2020-05-25/optimizer_type=RMS_use_all_words=False_emb_size=100_learning_rate=0.0001_word_embedding_type=mean/model/mean-inner-product-model.pt")
kwargs = dict(n_publications=len(final_publication_ids),
              n_articles=len(final_url_ids),
              n_attributes=len(final_word_ids),
              emb_size=100,
              sparse=False,
              use_article_emb=False,
              mode='mean')
model = InnerProduct(**kwargs)
model.load_state_dict(torch.load(abs_model_path))
print("Model Loaded.")

Dictionaries loaded.
Data loaded.
Final:  112060
Filtered, Mapped Data saved to \users\rohan\news-classification\Data\embedding-search\data\mapped-data directory
-------------------
Model Loaded.


In [ ]:
rows = []
cols = []
for idx, item in enumerate(raw_data.examples):
    word_ids = list(set(item['text']))
    number_of_words = np.arange(len(word_ids))
    rows.append(np.array(np.ones_like(number_of_words) * idx))
    cols.append(np.array(word_ids, dtype=np.int32))

In [122]:
final_rows = np.concatenate(rows, axis=None)
final_cols = np.concatenate(cols, axis=None)
final_data = np.ones_like(final_cols)

In [124]:
word_articles = csr_matrix((final_data, (final_rows, final_cols)))

In [126]:
print("Matrix Created!")

publication_emb = model.publication_embeddings.weight.data[0].cpu().numpy()
publication_bias = model.publication_bias.weight.data[0].cpu().numpy()
word_emb = model.attribute_emb_sum.weight.data.cpu().numpy()
word_bias = model.attribute_bias_sum.weight.data.cpu().numpy()

Matrix Created!


In [127]:
article_embeddings = word_articles.dot(word_emb)
word_counts = word_articles.sum(axis=1).reshape(word_articles.shape[0], 1)
print(article_embeddings.shape)
print(word_counts.shape)
article_embeddings = article_embeddings / word_counts

(112060, 100)
(112060, 1)


In [167]:
ranked = np.argsort(article_embeddings, axis=0)
ranked[1]

matrix([[ 13017,  13017,  13017,  32216,  13017,  13017, 110173, 111031,
          13017, 100152, 111889,  13017, 111031, 110173,  13017,  32216,
          32216, 110173,  13017,  13017,  13017, 111031,  13017, 102570,
         111031,  13017,  13017, 109602, 100152, 111806, 100152,  13017,
          32216, 100152, 100152, 102570, 110173, 100152,  13017, 111031,
          13017,  13017, 111031, 102570, 100152, 111806,  13017, 109602,
          13017, 110173, 111031, 110173, 100152,  13017,  13017, 110173,
         109602, 102570, 111031,  32216, 109602, 109602, 102570, 106597,
         100152,  13017,  13017, 100152, 111031, 109602, 109602, 110173,
         109602,  13017,  13017,  13017,  32216, 106597,  13017, 106597,
         100152, 110173, 102570, 109602,  13017,  13017,  13017,  13017,
          13017,  13017,  13017,  13017,  13017,  13017, 100152, 107710,
         100152,  13017,  13017, 111031]], dtype=int64)

In [163]:
for idx in range(100):
    current_iteration = ranked[:, idx][::-1]
    listed_iteration = np.squeeze(current_iteration).tolist()[0]
    print(idx)
    print("-----------------------------")
    for top in range(20):   
        print(listed_iteration[top], raw_data[listed_iteration[top]]['title'])
        print(article_embeddings[listed_iteration[top], idx])
    print("-----------------------------")

0
-----------------------------
102570 The Sewol Tragedy: Causes And Contributing Factors
0.09205894397261243
110173 Korean Politics 1: The Lay Of The Land
0.09205894397261243
111031 You Can Kill A Person With Three Punches
0.09205894397261243
109602 The Macroeconomics Of Chinese Kleptocracy
0.0812790748149262
100152 The Paulson Sino Forest Loss
0.0812790748149262
106597 The Intimate Orwell
0.0716509528986588
111889 Learning Curves
0.06937090916525028
111806 Queer Theory
0.06926168506636478
107710 How Europe Can Rescue Europe
0.06913596729564353
107569 A Few Thoughts
0.06907368479186204
111531 Grammar Yahtzee
0.0672200934753333
111427 Paperback to the Future
0.06624421822823394
64340 Gay Talese, The Art of Nonfiction No. 2
0.0645487648637814
97748 The Art of Nonfiction No. 2
0.0645487648637814
37758 The Geopolitical Consequences of the World Economic Recession -- A Caution
0.0644160137545407
108611 Do Cellphones Cause Brain Cancer?
0.06439823952336302
88529 Private and Public Supply of